## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that.

Fill in your **NAME** and **AEM** below:

In [1]:
NAME = "Vasiliki Gouma"
AEM = "9755"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [2]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file.
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not.


In [3]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x79c2ee681820>)

In [4]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

In [5]:
!pip install -U imbalanced-learn

The following code will reduce the number of instances, dealing with the small imbalance of the dataset, as well as reducing the size of the dataset!

In [6]:
from collections import Counter
from imblearn.under_sampling import NeighbourhoodCleaningRule, RandomUnderSampler

ncr = NeighbourhoodCleaningRule()
X_res, y_res = ncr.fit_resample(X, y)
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_res, y_res)
print('Resampled dataset shape %s' % Counter(y_res))
X = X_res
y = y_res

Resampled dataset shape Counter({np.int64(0): 1715, np.int64(1): 1715})


## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **5-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation.

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses two **simple** estimators/classifiers. Test both soft and hard voting and report the results. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   KNN Models  

In [7]:
### BEGIN SOLUTION

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold,cross_val_score



# USE RANDOM STATE!
cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE) # Classifier #1
cls2 = LogisticRegression(random_state=RANDOM_STATE,n_jobs=-1) # Classifier #2
soft_vcls = VotingClassifier(estimators=[('dt',cls1),('lg',cls2)],voting='soft',n_jobs=-1) # Voting Classifier
hard_vcls = VotingClassifier(estimators=[('dt',cls1),('lg',cls2)],voting='hard',n_jobs=-1) # Voting Classifier

kf = KFold(n_splits=5,shuffle=True,random_state=RANDOM_STATE)



scores1_s = cross_val_score(soft_vcls, X, y,scoring='f1_weighted',cv=kf,n_jobs=-1)
scores2_s = cross_val_score(soft_vcls, X, y,scoring='accuracy',cv=kf,n_jobs=-1)
scores1_h = cross_val_score(hard_vcls,X,y,scoring='f1_weighted',cv=kf,n_jobs=-1)
scores2_h = cross_val_score(hard_vcls,X,y,scoring='accuracy',cv=kf,n_jobs=-1)


print(scores1_s)
print(scores2_s)

print(scores1_h)
print(scores2_h)

iterations =5
# f1 svlcs_scores
svlcs_scores = list(scores1_s)
sm = sum(svlcs_scores[0:len(svlcs_scores)])
s_avg_fmeasure = sm/iterations # The average f-measure

# accuracy svlcs_scores
svlcs_scores = list(scores2_s)
sm = sum(svlcs_scores[0:len(svlcs_scores)])
s_avg_accuracy = sm/iterations # The average accuracy


# f1 hvlcs_scores
hvlcs_scores = list(scores1_h)
sm = sum(hvlcs_scores[0:len(svlcs_scores)])
h_avg_fmeasure = sm/iterations # The average f-measure

# accuracy hvlcs_scores
hvlcs_scores = list(scores2_h)
sm = sum(hvlcs_scores[0:len(svlcs_scores)])
h_avg_accuracy = sm/iterations # The average accuracy

### END SOLUTION

print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

[0.76096139 0.74493971 0.71706654 0.76344558 0.73032242]
[0.76093294 0.74489796 0.71720117 0.7638484  0.7303207 ]
[0.82711672 0.79525245 0.77050483 0.82185514 0.80280236]
[0.82944606 0.79883382 0.7755102  0.82361516 0.80612245]
Classifier:
VotingClassifier(estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                             ('lg',
                              LogisticRegression(n_jobs=-1, random_state=42))],
                 n_jobs=-1, voting='soft')
F1 Weighted-Score: 0.7433 & Balanced Accuracy: 0.7434
Classifier:
VotingClassifier(estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                             ('lg',
                              LogisticRegression(n_jobs=-1, random_state=42))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8035 & Balanced Accuracy: 0.8067


For both soft/hard voting classifiers the F1 weighted score should be above 0.74 and 0.79, respectively, and for balanced accuracy 0.74 and 0.80. Remember! This should be the average performance of each fold, as measured through cross-validation with 5 folds!

### 1.2 Randomization

You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements. Use the same cross-validation approach as before!

In [8]:
### BEGIN SOLUTION
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier


ens1 = BaggingClassifier(DecisionTreeClassifier(),n_jobs=-1,random_state=RANDOM_STATE)
ens2 = RandomForestClassifier(n_jobs=-1,random_state=RANDOM_STATE)
ens3 = XGBClassifier(tree_method='hist')
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

f_measures = dict()
accuracies = dict()

score_1f = cross_val_score(ens1,X,y,scoring='f1_weighted')
score=list(score_1f)
sm = sum(score[0:len(score)])
av = sm/iterations
f_measures['Bagging Classifier']=av

score_1a = cross_val_score(ens1,X,y,scoring='accuracy')
score=list(score_1a)
sm = sum(score[0:len(score)])
av = sm/iterations
accuracies['Bagging Classifier']=av

score_2f = cross_val_score(ens2,X,y,scoring='f1_weighted')
score=list(score_2f)
sm = sum(score[0:len(score)])
av = sm/iterations
f_measures['Random Forest Classifier']=av

score_2a = cross_val_score(ens2,X,y,scoring='accuracy')
score=list(score_2a)
sm = sum(score[0:len(score)])
av = sm/iterations
accuracies['Random Forest Classifier']=av

score_3f = cross_val_score(ens3,X,y,scoring='f1_weighted')
score=list(score_3f)
sm = sum(score[0:len(score)])
av = sm/iterations
f_measures['XGB Classifier']=av

score_3a = cross_val_score(ens3,X,y,scoring='accuracy')
score=list(score_3a)
sm = sum(score[0:len(score)])
av = sm/iterations
accuracies['XGB Classifier']=av

score_4f = cross_val_score(tree,X,y,scoring='f1_weighted')
score=list(score_4f)
sm = sum(score[0:len(score)])
av = sm/iterations
f_measures['Decision Tree classifier']=av

score_4a = cross_val_score(tree,X,y,scoring='accuracy')
score=list(score_4a)
sm = sum(score[0:len(score)])
av = sm/iterations
accuracies['Decision Tree Classifier']=av


# Example f_measures = {'Simple Decision':0.8551, 'Ensemble with random ...': 0.92, ...}


### END SOLUTION

print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier: {} -  F1 Weighted: {}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier: {} -  BalancedAccuracy: {}".format(name,round(score,4)))

BaggingClassifier(estimator=DecisionTreeClassifier(), n_jobs=-1,
                  random_state=42)
RandomForestClassifier(n_jobs=-1, random_state=42)
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)
DecisionTreeClassifier(random_state=42)
Classifier: Bagging Classifier -  F1 Weighted: 0

### 1.3 Question

Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

Bagging is a homogeneous weak learners model that learns from each other independently in parallel and combines them for determining the model average.So, since they can run parallel to each other, for more jobs (n_jobs increases) the training time is decreases because it run on more kernels.

This solution can not be applied to boosting classifiers.Boosting, that often considers homogeneous weak learners, learns them sequentially in a very adaptative way (a base model depends on the previous ones) and combines them following a deterministic strategy. This means that they can not run in parallel and therefore we can not aply this solution here.

## 2.0 Creating the best classifier ##
In the second part of this assignment, we will try to train the best classifier, as well as to evaluate it using stratified cross valdiation.

### 2.1 Good Performing Ensemble

In this part of the assignment you are asked to train a good performing ensemble, that is able to be used in a production environment! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy, using 10-fold stratified cross validation, of your final classifier. Can you achieve a balanced accuracy over 88%, while keeping the training time low? (Tip 1: You can even use a model from the previous parts, but you are advised to test additional configurations, and ensemble architectures, Tip 2: If you try a lot of models/ensembles/configurations or even grid searches, in your answer leave only the classifier you selected as the best!)

In [9]:
### BEGIN SOLUTION
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
import time

from xgboost import XGBClassifier
from sklearn.model_selection import KFold,cross_val_score




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=RANDOM_STATE)


ens10 = XGBClassifier(tree_method='hist')

scor = cross_val_score(ens10,X,y,scoring='f1_weighted')
score=list(scor)
sm = sum(score[0:len(score)])
av1 = sm/iterations

scor = cross_val_score(ens10,X,y,scoring='accuracy')
score=list(scor)
sm = sum(score[0:len(score)])
av2 = sm/iterations



start_time = time.time()
model = ens10.fit(X_train,y_train)
print(" %s seconds --- Training" % (round((time.time() - start_time),4)))



best_cls = ens10

best_fmeasure = av1
best_accuracy = av2
### END SOLUTION

print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(best_fmeasure, best_accuracy))

 42.5691 seconds --- Training
Classifier:
F1 Weighted-Score: 0.8775343636484875 & Balanced Accuracy: 0.8775510204081632


###Choosing the classifier:

I calculate the training time of the models from 1.3(BaggingClassifier
RandomForestClassifier,
XGBClassifier(tree_method='hist'),
DecisionTreeClassifier ) of the models from 1.1( VotingClassifier hard and soft) (I have already calculated the accuracy and f1 weighted score for these models).
Additionaly I calcutate accuracy,f1_weighted score, training time of the models StackingClassifier and  XGBClassifier(tree_method='hist_gpu').
The XGBClassifier (where tree_method='hist_gpu') is the method that I select because it has fairly low training time(~7 sec) and accuracy above 88% (88.94%)


In the description it is said that in our anwser we don't need to keep the code of all the models we tried (only the Classifier we chose), that is why i have deleted them from the code above.



### 2.2 Question
 What other ensemble architectures you tried, and why you did not choose them as your final classifier?

 I tried all the ensemble architectures mentioned in the text block above.The reason for not choosing them(except for XGBclassifier with tree_method='hist_gpu') is the high required training time and/or lower balanced accuracy than 88%.

YOUR ANSWER HERE

### 2.3 Setup the Final Classifier
Finally, in this last cell, set the cls variable to either the best model as occured by the stratified cross_validation, or choose to retrain your classifier in the whole dataset (X, y). There is no correct answer, but try to explain your choice. Then, save your model using pickle and upload it with your submission to e-learning!

In [12]:
import pickle

### BEGIN SOLUTION
cls = best_cls

# save with pickle
file_name = "xgb_classification.pkl"
pickle.dump(cls, open(file_name, "wb"))
### END SOLUTION


# load
cls = pickle.load(open(file_name, "rb"))

test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

# We are going to run the following code
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both metrics should aim above 82%! This is going to be tested by us! Make sure your cross validation or your retrained model achieves high balanced accuracy and f1_score (based on 2.1) (more than 88%) as it should achieve at least 82% in our unknown test set!


Please provide your feedback regarding this project! Did you enjoy it?

In [11]:
# YOUR ANSWER HERE
#Yepp!